In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["ANTHROPIC_API_KEY"] = os.getenv("ANTHROPIC_API_KEY")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.getenv("HUGGINGFACEHUB_API_TOKEN")

os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [ ]:
## Instantiation

from langchain_groq import ChatGroq
model=ChatGroq(model = "qwen-qwq-32b",temperature=1)
model.invoke("Hi My name is vivek") ## you can invoke it directly like this or use messages

In [ ]:
messages = [

    ("system","you are a product designer and should be able to answer product related queries"),
    ("human","What is a user journey"),
]

ai_msg = model.invoke(messages)
print(ai_msg.content)

In [ ]:
## Prompt engineering || Invocation

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","you are an expert AI engineer. Provide me answer based on the question in JSON format"),
        ("user","{input}")
    ]

)
prompt

In [ ]:
### Chaining
chain = prompt | model 
chain

In [ ]:
response = chain.invoke({"input":"can you tell me about langsmith"})
print(response.content)

In [ ]:
### STR Output Parser
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()
chain = prompt | model | output_parser
response = chain.invoke({"input":"can you tell me about langsmith"})


In [ ]:
print(response)

In [ ]:
### JSON Output Parser
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_groq import ChatGroq

output_parser = JsonOutputParser()

prompt = PromptTemplate(
    template = "Answer the user query \n {format_instruction} \n {query}",
    input_variables=["query"],
    partial_variables = {"format_instruction":output_parser.get_format_instructions()},
    )

model=ChatGroq(model = "gemma2-9b-it",temperature=0)

prompt


In [ ]:
chain = prompt|model|output_parser
response = chain.invoke({"query":"can you tell me about langsmith"})
print(response)

In [ ]:
### Assignement - Return the output in either JSON or XML using Chat prompt template

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_groq import ChatGroq

output_parser = JsonOutputParser()

prompt = ChatPromptTemplate (
    [
        ("system","You are an AI engineer, provide me the response in XML format"),
        ("human","{input}"),
    ]
)
prompt

model=ChatGroq(model = "gemma2-9b-it",temperature=0)
prompt


In [ ]:
chain = prompt|model
responses = chain.invoke({"input":"can you tell me about elon musk"})
print(responses)

In [ ]:
##Assignement - Return the ouptut using XML parser

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import XMLOutputParser
from langchain_groq import ChatGroq

output_parser = XMLOutputParser()

prompt = PromptTemplate(
    template = "Answer the user query in the following XML format:\n{format_instruction}\nQuery: {query}",
    input_variables=["query"],
    partial_variables = {"format_instruction":output_parser.get_format_instructions()},
    )

model=ChatGroq(model = "gemma2-9b-it",temperature=0)

prompt

In [ ]:
chain = prompt|model
response = chain.invoke({"query":"can you tell me about Elon musk"})
print(response)

In [17]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_groq import ChatGroq
from pydantic import BaseModel, Field

model=ChatGroq(model = "gemma2-9b-it",temperature=0.7)


# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")


# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a joke."

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

chain.invoke({"query": joke_query})

{'setup': "Why don't scientists trust atoms?",
 'punchline': 'Because they make up everything!'}

In [21]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_groq import ChatGroq
from pydantic import BaseModel, Field

model=ChatGroq(model = "gemma2-9b-it",temperature=0.7)


# Define your desired data structure.


# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a joke about dogs."

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser()

prompt = PromptTemplate(
    template="Answer the user query with two key value pairs setup and punchline.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

chain.invoke({"query": joke_query})

{'setup': 'Why do dogs run in circles?',
 'punchline': "Because it's really hard to run in squares!"}